In [1]:
import sys
import random
import pathlib
import pandas as pd

project_root = pathlib.Path.cwd().resolve()
while not (project_root / "backend").is_dir():
    if project_root.parent == project_root:
        raise RuntimeError("Could not find project root containing 'backend/'")
    project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from backend.utils import fetch_metrics
from backend import constants, data_retrieval


random_num = random.randint(0, 50)

country_data_df = pd.read_excel(r"/home/linux/Projects/AI-Country-Risk-Dashboard/backend/data/country_data.xlsx")
countries: list = country_data_df["Country_Name"].values.tolist()

# Testing Program for Random Country
selected_country = countries[random_num]
country_iso_code = country_data_df[country_data_df["Country_Name"] == selected_country]["iso2Code"].values.tolist()[0]

# Fetch All Data Points For A Random Country
country_data = fetch_metrics.build_country_panel(country_iso_code, constants.INDICATORS)
print(f"Country: {selected_country} | Country ISO Code: {country_iso_code}")
country_data

Country: Brazil | Country ISO Code: BR


,INFLATION,UNEMPLOYMENT,FDI_PCT_GDP,POL_STABILITY,RULE_OF_LAW,CONTROL_CORRUPTION,GINI_INDEX,GDP_PC_GROWTH,INT_PAYM_PCT_REV
1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.372339,NaN
1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.465911,NaN
1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.321402,NaN
1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.450645,NaN
...,...,...,...,...,...,...,...,...,...
2020,3.211768,13.697,2.592638,-0.446891,-0.231920,-0.429083,48.9,-3.835505,25.486547
2021,8.301660,13.158,2.779791,-0.419880,-0.301417,-0.499218,52.9,4.317932,27.098753
2022,9.280106,9.231,3.822196,-0.395037,-0.257879,-0.566354,52.0,2.646315,28.886777
2023,4.593563,7.947,2.849755,-0.408568,-0.308803,-0.503643,51.6,2.833700,30.839088


In [2]:
country_context = data_retrieval.prepare_llm_payload_pretty(
    country_iso=country_iso_code,
    indicators=constants.INDICATORS,
    since=2020,
    lookback=5
    )
country_context

{'country': 'BR',
 'latest_year': 2024,
 'indicators': {'Inflation (% y/y)': {'latest': 4.37,
   'Δ1y': -0.049,
   'Δ5y': None,
   'series': {2020: 3.21, 2021: 8.3, 2022: 9.28, 2023: 4.59, 2024: 4.37}},
  'Unemployment (% labour force)': {'latest': 7.63,
   'Δ1y': -0.039,
   'Δ5y': None,
   'series': {2020: 13.7, 2021: 13.16, 2022: 9.23, 2023: 7.95, 2024: 7.63}},
  'FDI inflow (% GDP)': {'latest': 3.26,
   'Δ1y': 0.144,
   'Δ5y': None,
   'series': {2020: 2.59, 2021: 2.78, 2022: 3.82, 2023: 2.85, 2024: 3.26}},
  'Political stability (z-score)': {'latest': None,
   'Δ1y': None,
   'Δ5y': None,
   'series': {2020: -0.45, 2021: -0.42, 2022: -0.4, 2023: -0.41}},
  'Rule of law (z-score)': {'latest': None,
   'Δ1y': None,
   'Δ5y': None,
   'series': {2020: -0.23, 2021: -0.3, 2022: -0.26, 2023: -0.31}},
  'Control of corruption (z-score)': {'latest': None,
   'Δ1y': None,
   'Δ5y': None,
   'series': {2020: -0.43, 2021: -0.5, 2022: -0.57, 2023: -0.5}},
  'Income inequality (Gini)': {'latest

In [3]:
"""
Example pipeline:
1. Pull structured macro data  ➜  prompt_points
2. Fetch fresh Google-News headlines
3. Ask the LLM for a risk score & summary
"""

import json
from datetime import timezone, datetime

from backend import constants
from backend import data_retrieval
from backend.ai import langchain_llm
from backend.utils import fetch_links

# ---------------------------------------------------------------------------
# 1. Build the macro-data payload (World Bank panel) -------------------------
# ---------------------------------------------------------------------------

COUNTRY_ISO   = country_iso_code            # Picked Country
RAW_INDICATORS = constants.INDICATORS       # {"INFLATION": "FP.CPI.TOTL.ZG", ...}

payload = data_retrieval.prepare_llm_payload_pretty(
    country_iso = COUNTRY_ISO,
    indicators  = RAW_INDICATORS,
    since       = 2015,
    lookback    = 10,
    deltas      = (1, 5)
)

# Turn the key → pretty-label mapping into the bullet points the prompt expects
prompt_points = ", ".join(payload["indicators"].keys())
# e.g. "Inflation (% y/y), Unemployment (% labour force), …"

# ---------------------------------------------------------------------------
# 2. Grab recent economic / political headlines -----------------------------
# ---------------------------------------------------------------------------

headlines = [
    item["title"]
    for item in fetch_links.gnews_rss(
        country  = selected_country,
        n        = 10,
        days     = 365,
        lang     = "en",
        region   = "US"
    )
]

# ---------------------------------------------------------------------------
# 3. Ask the LLM for a country risk score -----------------------------------
# ---------------------------------------------------------------------------

result = langchain_llm.country_llm_score(
    country       = payload["country"],
    headlines     = headlines,
    prompt_points = prompt_points
)

# ---------------------------------------------------------------------------
# 4. Combine & persist, ship to front-end, etc. -----------------------------
# ---------------------------------------------------------------------------

report = {
    **payload,                 # macro data
    "llm_output": result       # {"score": 0.72, "bullet_summary": "..."}
}

print(json.dumps(report, indent=2, ensure_ascii=False))

/home/linux/Projects/AI-Country-Risk-Dashboard/backend/ai/langchain_llm.py:70: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


{
  "country": "BR",
  "latest_year": 2024,
  "indicators": {
    "Inflation (% y/y)": {
      "latest": 4.37,
      "Δ1y": -0.049,
      "Δ5y": 0.17,
      "series": {
        "2015": 9.03,
        "2016": 8.74,
        "2017": 3.45,
        "2018": 3.66,
        "2019": 3.73,
        "2020": 3.21,
        "2021": 8.3,
        "2022": 9.28,
        "2023": 4.59,
        "2024": 4.37
      }
    },
    "Unemployment (% labour force)": {
      "latest": 7.63,
      "Δ1y": -0.039,
      "Δ5y": -0.36,
      "series": {
        "2015": 8.54,
        "2016": 11.58,
        "2017": 12.79,
        "2018": 12.33,
        "2019": 11.94,
        "2020": 13.7,
        "2021": 13.16,
        "2022": 9.23,
        "2023": 7.95,
        "2024": 7.63
      }
    },
    "FDI inflow (% GDP)": {
      "latest": 3.26,
      "Δ1y": 0.144,
      "Δ5y": -0.117,
      "series": {
        "2015": 3.59,
        "2016": 4.14,
        "2017": 3.34,
        "2018": 4.08,
        "2019": 3.69,
        "2020": 2.59